In [8]:
import os
import asyncio
from lightrag import LightRAG, QueryParam
from lightrag.llm.openai import gpt_4o_mini_complete, gpt_4o_complete, openai_embed
from lightrag.kg.shared_storage import initialize_pipeline_status
from lightrag.utils import setup_logger

setup_logger("lightrag", level="INFO")

In [9]:
WORKING_DIR = "light_rag_storage2"
if not os.path.exists(WORKING_DIR):
    os.mkdir(WORKING_DIR)

In [10]:
# import os, getpass
# if not os.environ.get("OPENAI_API_KEY"):
#   os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

In [ ]:
from lightrag.llm.openai import gpt_4o_mini_complete, openai_embed

    
async def initialize_rag():
    rag = LightRAG(
    working_dir=WORKING_DIR,
    llm_model_func=gpt_4o_mini_complete,
    embedding_func=openai_embed,  # <--- ADD THIS
    graph_storage="Neo4JStorage",
)
    # rag = LightRAG(
    #     working_dir=WORKING_DIR,
    #     embedding_func=openai_embed,
    #     llm_model_func=gpt_4o_mini_complete,
    # )
    # IMPORTANT: Both initialization calls are required!
    await rag.initialize_storages()  # Initialize storage backends
    await initialize_pipeline_status()  # Initialize processing pipeline
    return rag

In [ ]:
import os
os.environ["NEO4J_URI"] = "pass url here"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "pass password here"

In [ ]:
import nest_asyncio
import asyncio

async def main():
    try:
        # Initialize RAG instance
        rag = await initialize_rag()

        text = """
                pass text here
                """
        await rag.ainsert(text)

        # Perform hybrid search
        mode = "hybrid"
        print(
            await rag.aquery(
                "Explain Transformer?",
                param=QueryParam(mode=mode)
            )
        )

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        if rag:
            await rag.finalize_storages()

if __name__ == "__main__":
    nest_asyncio.apply()
    await main()
